In [1]:
#imports
import pandas as pd
import numpy as np
from collections import Counter
import re
import csv
import seaborn as sns
import matplotlib.pylab as plt

import warnings
warnings.filterwarnings('ignore')
import os

In [53]:
C_PATH = './country/'
countries = os.listdir(C_PATH)
countries.remove('results')
print(countries)

COUNTRY_LIST = []
for c in countries:
    c_code = c[:2]
    COUNTRY_LIST.append(c_code)
print(COUNTRY_LIST)

['bd.csv', 'tz.csv', 'au.csv']
['bd', 'tz', 'au']


In [54]:
TOPIC_LIST = ['ENVIRONMENT/ENERGY','INTERNATIONAL','POLITICS','EDUCATION/FAMILY', \
              'SPORTS', 'TECHNOLOGY/SCIENCE/SOCIAL MEDIA', 'SOCIAL LIFE/DAILY', \
              'ENTERTAINMENT/ART/MAGAZINE', 'COMPANY/BUSINESS', 'ECONOMY', \
              'POLICE/ACCIDENT/VIOLENCE', 'LEGAL/LAW','HEALTH/MEDICAL']

In [60]:
COUNTYR_DICT = {} ###### BY HAND ############
COUNTYR_DICT['tz'] = {0:{3:100}, 1:{0:100}, 2:{6:60, 7:40}, 3:{4:100}, 4:{5:100}, 5:{11:100}, 6:{8:50,12:50}} 
COUNTYR_DICT['bd'] = {0:{3:100}, 1:{0:100}, 2:{6:60, 7:30,8:10}, 3:{4:90,2:10}, 4:{5:100}, 5:{11:100}, 6:{8:50,12:50}}
COUNTYR_DICT['au'] = {0:{3:100}, 1:{0:50,1:50}, 2:{6:100}, 3:{4:90,2:10}, 4:{5:100}, 5:{11:100}, 6:{8:50,12:50}} 

In [56]:
for t, di in COUNTYR_DICT['tz'][2].items():
    print(t, di)
len(source_data)

6 60
7 40


6132163

## Read source data and asignment data

In [7]:
# Read news source folder as a whole
source_data = pd.read_table('now_sources_full.txt',encoding = "ISO-8859-1", header=None)
source_data.rename(columns={0:'textID', 1:'#words',2:'date',3:'country',4:'website',5:'url',6:'title'},inplace=True)
source_data.dropna(inplace=True)
source_data.date =  '20'+source_data.date
source_data.date =  pd.to_datetime(source_data.date, format='%Y-%m-%d')

In [57]:
def assign_percentage_topics(topic_frame, topic_dist):
    #print(topic_dist)
    #print(topic_frame)
    slices = []
    row_start = 0
    row_end = 0
    percen = 0
    for t_id, t_dis in topic_dist.items():
        
        percen = (percen+t_dis)/100
        row_end = row_start+  int(len(topic_frame)*percen)
        mini_slice = topic_frame[row_start:row_end]
        mini_slice.Topics = TOPIC_LIST[t_id]
        
        row_start = row_end
        
        #print(mini_slice)
        slices.append(mini_slice)
    
    return pd.concat(slices) 


In [58]:
def get_article_by_country(c_code, c_dict):
    assignment_data = pd.read_csv(C_PATH+ c_code +'.csv', header=None)
    assignment_data.rename(columns={0:'textID', 1:'Topics'},inplace=True)
    
    topic_frames = []
    for assign_topic in c_dict.keys():
        t_fr = assign_percentage_topics(assignment_data[assignment_data.Topics==assign_topic],\
                                        c_dict[assign_topic])
        topic_frames.append(t_fr)
                    ###assigment_data.replace({"Topics": c_dict}, inplace=True)
    return pd.concat(topic_frames)

In [61]:
c_frames = []

for cind in range(len(COUNTRY_LIST)) :
    c_code = COUNTRY_LIST[cind]
    c_dict = COUNTYR_DICT[c_code]
    per_country = get_article_by_country(c_code, c_dict)
    #per_country['country']=c_code.upper()
    c_frames.append(per_country)

all_counries = pd.concat(c_frames)

In [12]:
#tz_data = pd.read_csv(C_PATH+ 'tz' +'.csv', header=None)
#tz_data.rename(columns={0:'textID', 1:'Topics'},inplace=True)
# assignment_data[assignmnent_data.Topics==2].Topics.value_counts()

In [62]:
all_counries.head()

,textID,Topics
5,14064675,EDUCATION/FAMILY
28,13937480,EDUCATION/FAMILY
30,14275019,EDUCATION/FAMILY
33,15100156,EDUCATION/FAMILY
34,10173097,EDUCATION/FAMILY


In [63]:
full_data = pd.merge(source_data, all_counries, left_on='textID', right_on='textID')

In [64]:
full_data

,textID,#words,date,country,website,url,title,Topics
0,3735097,736,2015-11-01,AU,Sydney Morning Herald,http://www.smh.com.au/entertainment/books/judy...,Judy Nunn and Spirits of the Ghan,EDUCATION/FAMILY
1,3735099,319,2015-11-01,AU,ABC Online,http://www.abc.net.au/news/2015-11-02/rugby-wo...,Coach of the Year - Michael Cheika (Australia),INTERNATIONAL
2,3735107,804,2015-11-01,AU,Sydney Morning Herald,http://www.smh.com.au/environment/csiro-teams-...,CSIRO team's study erodes credibility of key s...,LEGAL/LAW
3,3735110,652,2015-11-01,AU,ABC Online,http://www.abc.net.au/news/2015-11-02/life-as-...,In the shadow of death: life as a palliative c...,SPORTS
4,3735111,192,2015-11-01,AU,ABC Online,http://www.abc.net.au/news/2015-11-02/15-years...,Fifteen years of humans living on Internationa...,COMPANY/BUSINESS
5,3735166,305,2015-11-01,AU,The Canberra Times,http://www.canberratimes.com.au/sport/act-spor...,Heavy rain and lightning forces postponement o...,INTERNATIONAL
6,3735196,792,2015-11-01,AU,Gizmodo Australia,http://www.gizmodo.com.au/2015/11/this-feature...,This Feature Film Is Billing An Android As One...,EDUCATION/FAMILY
7,3735206,1022,2015-11-01,AU,Motley Fool Australia,http://www.fool.com.au/2015/11/02/2-bargain-bl...,2 bargain blue chips? Commonwealth Bank of Aus...,LEGAL/LAW
8,3735207,217,2015-11-01,AU,Pedestrian TV,http://www.pedestrian.tv/news/sport/wallaby-da...,Wallaby David Pocock Accused Of Stomping Rival...,EDUCATION/FAMILY
9,3735212,280,2015-11-01,AU,ABC Online,http://blogs.abc.net.au/queensland/2015/11/who...,Who Are You? Bob East - CEO of Gold Coast base...,LEGAL/LAW
